# Lost & Found for colby
- recognize everyday objects students commonly lose — like phones, wallets, AirPods, glasses, cards, and more ​

- to automatically categorize them into a searchable website interface.​

- So that you don’t have to walk in snow to check whether your lost items is in the lost and found center!​

In [3]:
!pip install ultralytics roboflow opencv-python pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 107.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [12]:
from google.colab import drive; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os, sys

project_dir = "/content/drive/MyDrive/Colab Notebooks/lost_found_project"
os.chdir(project_dir)
sys.path.append(project_dir)

!ls -lh


total 42K
-rw------- 1 root root  14K Dec  1 08:44 'Computer vision final project - Ariel.ipynb'
drwx------ 2 root root 4.0K Dec  1 08:37  converted_images
-rw------- 1 root root 8.2K Dec  1 08:19  dataset_preparation.py
drwx------ 2 root root 4.0K Dec  1 08:01  __pycache__
drwx------ 2 root root 4.0K Dec  1 06:21  raw_images
drwx------ 2 root root 4.0K Dec  1 08:01  resized_images
drwx------ 2 root root 4.0K Dec  1 08:34  validated_images


In [4]:
!pip install roboflow ultralytics pillow pyyaml -q

In [24]:
from dataset_preparation import resize_images, merge_yolo_datasets, verify_dataset, visualize_samples


import os
import shutil
import yaml
from PIL import Image, ImageDraw, ImageFile
import matplotlib.pyplot as plt
from google.colab import files

# Allow loading truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [25]:
import importlib
import dataset_preparation
importlib.reload(dataset_preparation)

from dataset_preparation import resize_images, merge_yolo_datasets, verify_dataset, visualize_samples

In [26]:
# Allow loading truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Install pillow-heif for HEIC support
print("Installing pillow-heif...")
os.system('pip install pillow-heif -q')

from pillow_heif import register_heif_opener
register_heif_opener()

# Define paths
raw_images_path = "/content/drive/MyDrive/Colab Notebooks/lost_found_project/raw_images"
converted_path = "/content/drive/MyDrive/Colab Notebooks/lost_found_project/converted_images"
resized_output_path = "/content/drive/MyDrive/Colab Notebooks/lost_found_project/resized_images"

# Create output directory
os.makedirs(converted_path, exist_ok=True)

# Convert HEIC to JPG
print("\n Converting HEIC to JPG")
converted = 0
copied = 0
failed = []

for img_name in os.listdir(raw_images_path):
    if not img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.heic', '.heif')):
        continue

    img_path = os.path.join(raw_images_path, img_name)
    base_name = os.path.splitext(img_name)[0]
    output_name = f"{base_name}.jpg"
    output_path = os.path.join(converted_path, output_name)

    try:
        img = Image.open(img_path)

        if img.mode != 'RGB':
            img = img.convert('RGB')

        img.save(output_path, 'JPEG', quality=95)

        if img_name.lower().endswith(('.heic', '.heif')):
            converted += 1
            print(f"Converted: {img_name}")
        else:
            copied += 1

    except Exception as e:
        failed.append((img_name, str(e)))
        print(f"Failed: {img_name}")

print(f"\nConverted: {converted}, Copied: {copied}, Failed: {len(failed)}")

# Resize images using function from dataset_preparation
print("\nResizing images to (640,640)")
num_processed = resize_images(converted_path, resized_output_path, target_size=(640, 640))

print(f"\nTotal processed: {num_processed}")

# Create ZIP for Roboflow
print("\nCreating ZIP file")
zip_path = '/content/resized_images_for_roboflow'
shutil.make_archive(zip_path, 'zip', resized_output_path)
files.download(f'{zip_path}.zip')


Installing pillow-heif...

 Converting HEIC to JPG

Converted: 0, Copied: 44, Failed: 0

Resizing images to (640,640)
Processed: new place 🔑 #саморазвитие #психология….jpg -> img_0000.jpg
Processed: Keys.jpg -> img_0001.jpg
Processed: truth.jpg -> img_0002.jpg
Processed: New house keys _ Premium Photo.jpg -> img_0003.jpg
Processed: The Key You Can’t Ask For.jpg -> img_0004.jpg
Processed: House key on plans stock photo_ Image of plans, extension - 21591266.jpg -> img_0005.jpg
Processed: Vintage _Random Lockers_ Key Set - (#18 Total).jpg -> img_0006.jpg
Processed: KEYS_2929.jpg -> img_0007.jpg
Processed: KEYS_2928.jpg -> img_0008.jpg
Processed: KEYS_2927.jpg -> img_0009.jpg
Processed: KEYS_2926.jpg -> img_0010.jpg
Processed: KEYS_2925.jpg -> img_0011.jpg
Processed: KEYS_2924.jpg -> img_0012.jpg
Processed: KEYS_2923.jpg -> img_0013.jpg
Processed: KEYS_2922.jpg -> img_0014.jpg
Processed: keys (1).jpg -> img_0015.jpg
Processed: IMG_2957.jpg -> img_0016.jpg
Processed: IMG_2956.jpg -> img_001

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
import os
import shutil
from dataset_preparation import merge_yolo_datasets, verify_dataset, visualize_samples

# Install roboflow
os.system('pip install roboflow -q')

from roboflow import Roboflow

In [54]:
print("Downloading AirPods dataset...")
rf = Roboflow(api_key="tSpyQRRhpZElHqZ7Y81V")
project = rf.workspace("project-uzwxb").project("airpods-7mme5")
version = project.version(7)
dataset1 = version.download("yolov8")
dataset1_path = dataset1.location
print(f"Dataset 1 downloaded to: {dataset1_path}")

# Download Dataset 2: Battery/Charger/Cable
print("\nDownloading Battery/Charger/Cable dataset...")
rf = Roboflow(api_key="tSpyQRRhpZElHqZ7Y81V")
project = rf.workspace("boomaie").project("battery-charger-cable-82kib")
version = project.version(2)
dataset2 = version.download("yolov8")
dataset2_path = dataset2.location
print(f"Dataset 2 downloaded to: {dataset2_path}")

# Download Dataset 3: My custom annotated dataset (mainly Colby cards)
print("\nDownloading my custom dataset...")
rf = Roboflow(api_key="tSpyQRRhpZElHqZ7Y81V")
project = rf.workspace("yilin-pan-gzs5y").project("lostandfind-06kgb")
version = project.version(1)
dataset3 = version.download("yolov8")
dataset3_path = dataset3.location
print(f"Dataset 3 downloaded to: {dataset3_path}")

# Download Dataset 4: Scarfs
print("\nDownloading Scarfs dataset...")
rf = Roboflow(api_key="tSpyQRRhpZElHqZ7Y81V")
project = rf.workspace("practicas-6jgbv").project("scarf-zkmzt")
version = project.version(1)
dataset4 = version.download("yolov8")
dataset4_path = dataset4.location
print(f"Dataset 4 downloaded to: {dataset4_path}")

# Download Dataset 5: Jackets/Vests
print("\nDownloading Jackets/Vests dataset...")
rf = Roboflow(api_key="tSpyQRRhpZElHqZ7Y81V")
project = rf.workspace("meister1").project("jackets-vests")
version = project.version(1)
dataset5 = version.download("yolov8")
dataset5_path = dataset5.location
print(f"Dataset 5 downloaded to: {dataset5_path}")

# Doanload Dataset 6: General Lost and Found
print("\nDownloading general lost and fonud dataset...")
rf = Roboflow(api_key="tSpyQRRhpZElHqZ7Y81V")
project = rf.workspace("mercys-workspace-7ti14").project("lost-and-found-items")
version = project.version(7)
dataset6 = version.download("yolov8")
dataset6_path = dataset6.location
print(f"Dataset 6 downloaded to: {dataset6_path}")



loading Roboflow workspace...
loading Roboflow project...
Dataset 1 downloaded to: /content/drive/MyDrive/Colab Notebooks/lost_found_project/Airpods-7

loading Roboflow workspace...
loading Roboflow project...
Dataset 2 downloaded to: /content/drive/MyDrive/Colab Notebooks/lost_found_project/Battery-charger-cable-2

loading Roboflow workspace...
loading Roboflow project...
Dataset 3 downloaded to: /content/drive/MyDrive/Colab Notebooks/lost_found_project/LostandFind-1

loading Roboflow workspace...
loading Roboflow project...
Dataset 4 downloaded to: /content/drive/MyDrive/Colab Notebooks/lost_found_project/Scarf-1

loading Roboflow workspace...
loading Roboflow project...
Dataset 5 downloaded to: /content/drive/MyDrive/Colab Notebooks/lost_found_project/jackets-vests-1

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Lost-and-found-items-7 in yolov8:: 100%|██████████| 4358/4358 [00:37<00:00, 116.35it/s]

Dataset 6 downloaded to: /content/drive/MyDrive/Colab Notebooks/lost_found_project/Lost-and-found-items-7


In [48]:
import yaml
import os

def verify_dataset_fixed(dataset_path):
    """Verify dataset structure and print statistics"""
    print(f"\nVerifying: {dataset_path}\n")

    stats = {
        'train_images': 0,
        'train_labels': 0,
        'valid_images': 0,
        'valid_labels': 0,
        'test_images': 0,
        'test_labels': 0,
        'classes': {}
    }

    for split in ['train', 'valid', 'test']:
        img_dir = f'{dataset_path}/{split}/images'
        lbl_dir = f'{dataset_path}/{split}/labels'

        if os.path.exists(img_dir):
            stats[f'{split}_images'] = len(os.listdir(img_dir))
        if os.path.exists(lbl_dir):
            stats[f'{split}_labels'] = len(os.listdir(lbl_dir))

    yaml_path = f'{dataset_path}/data.yaml'
    if os.path.exists(yaml_path):
        with open(yaml_path, 'r') as f:
            data = yaml.safe_load(f)
        stats['classes'] = data['names']

    # Handle both dict and list formats
    if isinstance(stats['classes'], dict):
        class_list = list(stats['classes'].values())
    else:
        class_list = stats['classes']

    print("Dataset Summary:")
    print(f"  Train: {stats['train_images']} images, {stats['train_labels']} labels")
    print(f"  Valid: {stats['valid_images']} images, {stats['valid_labels']} labels")
    print(f"  Test: {stats['test_images']} images, {stats['test_labels']} labels")
    print(f"  Classes: {class_list}")

    if stats['train_images'] != stats['train_labels']:
        print(f"\nWarning: Train images/labels mismatch")
    if stats['valid_images'] != stats['valid_labels']:
        print(f"Warning: Valid images/labels mismatch")

    return stats


In [49]:
def merge_yolo_datasets_with_remap(dataset_paths, output_path, dataset_prefixes=None):

    if dataset_prefixes is None:
        dataset_prefixes = [f"ds{i}" for i in range(len(dataset_paths))]

    # Create output structure
    for split in ['train', 'valid', 'test']:
        os.makedirs(f'{output_path}/{split}/images', exist_ok=True)
        os.makedirs(f'{output_path}/{split}/labels', exist_ok=True)

    # Collect all unique classes across datasets
    all_class_names = []
    class_mappings = []

    for dataset_path in dataset_paths:
        yaml_path = f'{dataset_path}/data.yaml'
        with open(yaml_path, 'r') as f:
            data = yaml.safe_load(f)

        dataset_classes = data['names']
        if isinstance(dataset_classes, dict):
            dataset_classes = list(dataset_classes.values())

        print(f"  {dataset_path}: {dataset_classes}")

        # Create mapping for this dataset
        old_to_new = {}
        for old_id, class_name in enumerate(dataset_classes):
            if class_name not in all_class_names:
                all_class_names.append(class_name)

            # Map old ID to new ID
            new_id = all_class_names.index(class_name)
            old_to_new[old_id] = new_id

        class_mappings.append(old_to_new)

    # Convert class names list to dict
    all_classes = {i: name for i, name in enumerate(all_class_names)}
    print(f"\nMerged classes: {all_classes}")

    # Process each dataset
    for dataset_idx, (dataset_path, prefix) in enumerate(zip(dataset_paths, dataset_prefixes)):
        print(f"\nProcessing dataset: {dataset_path}")
        class_mapping = class_mappings[dataset_idx]

        for split in ['train', 'valid', 'test']:
            img_dir = f'{dataset_path}/{split}/images'
            lbl_dir = f'{dataset_path}/{split}/labels'

            if not os.path.exists(img_dir):
                continue

            # Copy images
            img_count = 0
            for img in os.listdir(img_dir):
                new_name = f"{prefix}_{img}"
                shutil.copy(
                    os.path.join(img_dir, img),
                    f'{output_path}/{split}/images/{new_name}'
                )
                img_count += 1

            # Copy and remap labels
            lbl_count = 0
            if os.path.exists(lbl_dir):
                for lbl in os.listdir(lbl_dir):
                    lbl_path = os.path.join(lbl_dir, lbl)

                    # Read and remap labels
                    with open(lbl_path, 'r') as f:
                        lines = f.readlines()

                    remapped_lines = []
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) < 5:
                            continue

                        old_class_id = int(parts[0])
                        new_class_id = class_mapping.get(old_class_id, old_class_id)
                        parts[0] = str(new_class_id)
                        remapped_lines.append(' '.join(parts) + '\n')

                    # Write remapped labels
                    new_name = f"{prefix}_{lbl}"
                    with open(f'{output_path}/{split}/labels/{new_name}', 'w') as f:
                        f.writelines(remapped_lines)
                    lbl_count += 1

            print(f"  {split}: {img_count} images, {lbl_count} labels")

    # Create merged data.yaml
    merged_yaml = {
        'path': output_path,
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'names': all_classes
    }

    with open(f'{output_path}/data.yaml', 'w') as f:
        yaml.dump(merged_yaml, f)

    print(f"\nMerged dataset created at: {output_path}")
    print(f"Total classes: {len(all_classes)}")
    print(f"Classes: {list(all_classes.values())}")

    return output_path

In [55]:
import random
# Dataset paths
dataset_paths = [
    dataset1_path,  # airpods
    dataset2_path,  # battery-charger-cable
    dataset3_path,  # custom lost&found
    dataset4_path,  # scarf
    dataset5_path,   # jackets-vests
    dataset6_path  # general lost-and-found
  ]

dataset_prefixes = ['airpods', 'battery', 'custom', 'scarf', 'jackets','lostandfound']

# Where to save the merged dataset
output_path = '/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset'

# Classes to exclude (there's some issue with my own airpods annotation, we will be using downloaded airpods images instead)
exclude_classes = ['airpods', 'airpods-whole', 'none']

# Clean output
if os.path.exists(output_path):
    shutil.rmtree(output_path)

for split in ['train', 'valid', 'test']:
    os.makedirs(f'{output_path}/{split}/images', exist_ok=True)
    os.makedirs(f'{output_path}/{split}/labels', exist_ok=True)


all_class_names = []

print("\nCollecting classes:\n")
for dataset_path in dataset_paths:
    yaml_path = os.path.join(dataset_path, 'data.yaml')
    if not os.path.exists(yaml_path):
        print(f"  Warning: {yaml_path} not found, skipping.")
        continue

    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    classes = data['names']
    if isinstance(classes, dict):
        classes = list(classes.values())

    print(f"{os.path.basename(dataset_path)}: {classes}")

    for class_name in classes:
        if class_name not in exclude_classes and class_name not in all_class_names:
            all_class_names.append(class_name)

print(f"\nFinal classes (after filtering): {all_class_names}\n")

# Create ID mappings from each dataset to the unified class list

class_mappings = []
for dataset_path in dataset_paths:
    yaml_path = os.path.join(dataset_path, 'data.yaml')
    if not os.path.exists(yaml_path):
        class_mappings.append({})
        continue

    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    classes = data['names']
    if isinstance(classes, dict):
        classes = list(classes.values())

    mapping = {}
    for old_id, class_name in enumerate(classes):
        if class_name in all_class_names:
            mapping[old_id] = all_class_names.index(class_name)

    class_mappings.append(mapping)

# Copy and filter files for each dataset and split

for dataset_idx, (dataset_path, prefix) in enumerate(zip(dataset_paths, dataset_prefixes)):
    print(f"Processing {prefix} from {dataset_path} ...")
    mapping = class_mappings[dataset_idx]

    for split in ['train', 'valid', 'test']:
        img_dir = os.path.join(dataset_path, split, 'images')
        lbl_dir = os.path.join(dataset_path, split, 'labels')

        if not os.path.exists(img_dir):
            continue

        for img_name in os.listdir(img_dir):
            if not img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
                continue

            lbl_name = os.path.splitext(img_name)[0] + '.txt'
            lbl_path = os.path.join(lbl_dir, lbl_name)

            if not os.path.exists(lbl_path):
                continue

            # Read and filter labels
            with open(lbl_path, 'r') as f:
                lines = f.readlines()

            remapped = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    old_id = int(parts[0])
                    if old_id in mapping:
                        parts[0] = str(mapping[old_id])
                        remapped.append(' '.join(parts) + '\n')

            # Only copy if we have valid labels after filtering
            if remapped:
                shutil.copy(
                    os.path.join(img_dir, img_name),
                    os.path.join(output_path, split, 'images', f'{prefix}_{img_name}')
                )

                with open(os.path.join(output_path, split, 'labels', f'{prefix}_{lbl_name}'), 'w') as f:
                    f.writelines(remapped)

# Create validation split from training data

print("\nCreating validation split from training data...")
train_img_dir = os.path.join(output_path, 'train', 'images')
train_lbl_dir = os.path.join(output_path, 'train', 'labels')

all_train_images = os.listdir(train_img_dir)
num_train = len(all_train_images)
num_valid = int(num_train * 0.15)

random.shuffle(all_train_images)
valid_images = all_train_images[:num_valid]

for img_name in valid_images:
    lbl_name = os.path.splitext(img_name)[0] + '.txt'

    shutil.move(
        os.path.join(train_img_dir, img_name),
        os.path.join(output_path, 'valid', 'images', img_name)
    )

    lbl_src = os.path.join(train_lbl_dir, lbl_name)
    if os.path.exists(lbl_src):
        shutil.move(
            lbl_src,
            os.path.join(output_path, 'valid', 'labels', lbl_name)
        )

# Create final data.yaml
final_classes = {i: name for i, name in enumerate(all_class_names)}

yaml_content = {
    'path': output_path,
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'names': final_classes
}

with open(os.path.join(output_path, 'data.yaml'), 'w') as f:
    yaml.dump(yaml_content, f)

# Verify
print(f"\nDataset created at: {output_path}")
print(f"Classes ({len(final_classes)}): {list(final_classes.values())}")

train_count = len(os.listdir(os.path.join(output_path, 'train', 'images')))
valid_count = len(os.listdir(os.path.join(output_path, 'valid', 'images')))
total = train_count + valid_count

print(f"\nTrain: {train_count} ({train_count / total * 100:.1f}%)")
print(f"Valid: {valid_count} ({valid_count / total * 100:.1f}%)")
print(f"Total: {total}")



Airpods-7: ['Airpods']
Battery-charger-cable-2: ['c to c', 'c toc', 'lightning to c', 'usb in 3', 'usb to  LIghtning', 'usb to c', 'usb to micro-usb']
LostandFind-1: ['airpods', 'airpods-whole', 'colby student card', 'keys', 'none', 'powerbank']
Scarf-1: ['Scarf']
jackets-vests-1: ['jackets', 'vests']
Lost-and-found-items-7: ['Backpack', 'Belt-bag', 'Car-key-Acura', 'Car-key-Chevrolet', 'Car-key-Ford', 'Car-key-Honda', 'Car-key-Hyundai', 'Car-key-Kia', 'Car-key-Lexus', 'Car-key-Mercedes-Benz', 'Car-key-Mitsubishi', 'Car-key-Nissan', 'Car-key-Toyota', 'Car-key-Volkswagen', 'Car-key-non-specific', 'Card', 'Cross-bag', 'Electronic-devices-AirPods', 'Electronic-devices-AirPods-case', 'Electronic-devices-charger', 'Electronic-devices-earphones', 'Electronic-devices-headphones', 'Electronic-devices-laptop', 'Electronic-devices-laptop-charger', 'Electronic-devices-phone', 'Electronic-devices-phone-charger', 'Electronic-devices-powerbank', 'Electronic-devices-tablet', 'Glasses', 'Handbag', '

In [86]:
import os
import yaml
from collections import defaultdict

dataset_path = output_path


with open(f"{dataset_path}/data.yaml", "r") as f:
    data = yaml.safe_load(f)

classes = data["names"]
if isinstance(classes, dict):
    classes = list(classes.values())

def group_class(class_name):
    lower_name = class_name.lower()

    if " to " in lower_name or "to" in lower_name.split("-") or "cable" in lower_name or "toc" in lower_name or "usb" in lower_name:
        return "cables"

    if "car-key" in lower_name or "car key" in lower_name:
        return "car-keys"

    if "watch" in lower_name:
        if "smart" in lower_name:
            return "smart-watches"
        else:
            return "watches"

    if "wallet" in lower_name:
        return "wallets"

    if "phone" in lower_name:
        return "phones"

    if "airpod" in lower_name:
        return "airpods"

    if "tablet" in lower_name:
        return "tablets"

    if "glasses" in lower_name or "sunglasses" in lower_name:
        return "glasses"

    if "glove" in lower_name:
        return "gloves"

    if "powerbank" in lower_name:
        return "powerbank"


    if "laptop" in lower_name or "charger" in lower_name:
        if "charger" in lower_name:
            return "chargers"
        else:
            return "laptops"
    if "bag" in lower_name:
      return "bags(other than backpack)"

    if "keys" in lower_name or "key" in lower_name:
      return "keys"

    return class_name


print("=" * 60)
print("image distribution by grouped label")
print("=" * 60)

group_image_counts = defaultdict(int)
total_images = 0

for split in ["train", "valid", "test"]:
    lbl_dir = os.path.join(dataset_path, split, "labels")
    if not os.path.exists(lbl_dir):
        continue

    for lbl_file in os.listdir(lbl_dir):
        if not lbl_file.endswith(".txt"):
            continue

        total_images += 1
        img_groups = set()   # groups that appear in this image

        with open(os.path.join(lbl_dir, lbl_file), "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 5:
                    continue
                cls_id = int(parts[0])
                if 0 <= cls_id < len(classes):
                    cls_name = classes[cls_id]
                    g = group_class(cls_name)
                    img_groups.add(g)

        # count this image once for each group it contains
        for g in img_groups:
            group_image_counts[g] += 1

print(f"\nTotal images (train+valid+test): {total_images}\n")

for g, count in sorted(group_image_counts.items(),
                       key=lambda x: x[1],
                       reverse=True):
    pct = count / total_images * 100 if total_images > 0 else 0
    print(f"{g}: {count} images ({pct:.1f}%)")


image distribution by grouped label

Total images (train+valid+test): 3337

phones: 488 images (14.6%)
jackets: 381 images (11.4%)
glasses: 342 images (10.2%)
airpods: 270 images (8.1%)
keys: 231 images (6.9%)
Scarf: 200 images (6.0%)
cables: 198 images (5.9%)
watches: 197 images (5.9%)
bags(other than backpack): 176 images (5.3%)
wallets: 174 images (5.2%)
Card: 135 images (4.0%)
car-keys: 131 images (3.9%)
Backpack: 109 images (3.3%)
laptops: 103 images (3.1%)
powerbank: 53 images (1.6%)
colby student card: 45 images (1.3%)
tablets: 39 images (1.2%)
chargers: 37 images (1.1%)
smart-watches: 26 images (0.8%)
vests: 18 images (0.5%)


In [88]:
import os
import shutil
import yaml


dataset_path = '/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset'

# slimmed dataset
slim_dataset_path = dataset_path + "_slim"
os.makedirs(slim_dataset_path, exist_ok=True)


with open(os.path.join(dataset_path, "data.yaml"), "r") as f:
    data = yaml.safe_load(f)

orig_classes = data["names"]
if isinstance(orig_classes, dict):
    orig_classes = [orig_classes[i] for i in sorted(orig_classes.keys())]


group_names = [
    "phones",
    "jackets",
    "glasses",
    "airpods",
    "Scarf",
    "cables",
    "watches",
    "bags(other than backpack)",
    "wallets",
    "card",
    "car-keys",
    "Backpack",
    "laptops",
    "keys",
    "powerbank",
    "colby student card",
    "tablets",
    "chargers",
    "smart-watches",
    "vests",
]
group2id = {name: i for i, name in enumerate(group_names)}


def group_class(class_name: str):
    lower = class_name.lower()

    # Phones, laptops, tablets
    if "phone" in lower:
        return "phones"
    if "tablet" in lower:
        return "tablets"
    if "laptop" in lower:
        return "laptops"

    # Jackets, vests, scarves
    if "jacket" in lower:
        return "jackets"
    if "vest" in lower:
        return "vests"
    if "scarf" in lower:
        return "Scarf"

    # Glasses & sunglasses
    if "glasses" in lower or "sunglasses" in lower:
        return "glasses"

    # AirPods, earbuds, earphones
    if "airpod" in lower or "earphone" in lower or "earbud" in lower:
        return "airpods"

    # Cables & chargers & powerbank
    if "usb" in lower or "c to c" in lower or "c-to-c" in lower or "cable" in lower or " to " in lower:
        return "cables"
    if "charger" in lower:
        return "chargers"
    if "powerbank" in lower:
        return "powerbank"

    # Watches
    if "smart-watch" in lower.replace("_", "-"):
        return "smart-watches"
    if "watch" in lower:
        return "watches"

    # Bags and backpack
    if "backpack" in lower:
        return "Backpack"
    if "bag" in lower:
        return "bags(other than backpack)"

    # Wallets
    if "wallet" in lower:
        return "wallets"

    # Cards
    if "colby" in lower and "card" in lower:
        return "colby student card"
    if "card" in lower:
        return "card"

    # Keys & car-keys
    if "car-key" in lower or "car key" in lower:
        return "car-keys"
    if "keys" in lower or "key" in lower:
        return "keys"

    # Classes not matched will be removed
    return None

# Rewrite labels & copy images
splits = ["train", "valid", "test"]

for split in splits:
    img_src_dir = os.path.join(dataset_path, split, "images")
    lbl_src_dir = os.path.join(dataset_path, split, "labels")

    if not os.path.exists(img_src_dir) or not os.path.exists(lbl_src_dir):
        print(f"split '{split}' not found, skipping.")
        continue

    img_dst_dir = os.path.join(slim_dataset_path, split, "images")
    lbl_dst_dir = os.path.join(slim_dataset_path, split, "labels")
    os.makedirs(img_dst_dir, exist_ok=True)
    os.makedirs(lbl_dst_dir, exist_ok=True)

    for lbl_file in os.listdir(lbl_src_dir):
        if not lbl_file.endswith(".txt"):
            continue

        src_lbl_path = os.path.join(lbl_src_dir, lbl_file)
        dst_lbl_path = os.path.join(lbl_dst_dir, lbl_file)

        with open(src_lbl_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue

            cls_id = int(parts[0])
            if not (0 <= cls_id < len(orig_classes)):
                continue

            orig_name = orig_classes[cls_id]
            gname = group_class(orig_name)
            if gname is None:
                continue

            if gname not in group2id:
                print(f"grouped name '{gname}' not in group_names (orig '{orig_name}')")
                continue

            new_id = group2id[gname]
            new_line = " ".join([str(new_id)] + parts[1:])
            new_lines.append(new_line + "\n")

        # Only keep images that have at least 1 valid object
        if new_lines:
            with open(dst_lbl_path, "w") as f:
                f.writelines(new_lines)

            base = os.path.splitext(lbl_file)[0]
            found = False
            for ext in [".jpg", ".jpeg", ".png"]:
                src_img_path = os.path.join(img_src_dir, base + ext)
                if os.path.exists(src_img_path):
                    shutil.copy2(src_img_path, os.path.join(img_dst_dir, base + ext))
                    found = True
                    break
            if not found:
                print(f"[WARN] image for {lbl_file} not found in {img_src_dir}")

print("Slim dataset written to:", slim_dataset_path)

# data_slim.yaml
new_data = {
    "path": slim_dataset_path,
    "train": os.path.join(slim_dataset_path, "train", "images"),
    "val": os.path.join(slim_dataset_path, "valid", "images"),
    "test": os.path.join(slim_dataset_path, "test", "images"),
    "nc": len(group_names),
    "names": {i: name for i, name in enumerate(group_names)},
}

with open(os.path.join(slim_dataset_path, "data_slim.yaml"), "w") as f:
    yaml.safe_dump(new_data, f, sort_keys=False)

print("New data file saved to:", os.path.join(slim_dataset_path, "data_slim.yaml"))


Slim dataset written to: /content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim
New data file saved to: /content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim/data_slim.yaml


In [75]:
import os
import shutil
os.system('pip install ultralytics -q')

from ultralytics import YOLO
from IPython.display import Image, display

print("Setup complete")

Setup complete


In [91]:

import os
import shutil
from ultralytics import YOLO
import torch

from ultralytics import YOLO

# Load COCO-pretrained YOLO model nano
model = YOLO("yolov8n.pt")

# Fine-tune on slimmed dataset
results = model.train(
    data="/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim/data_slim.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    workers=4,
    project="lostfound_slim_exp",
    name="yolov8s_slim_finetune",
    freeze = 10
)


Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim/data_slim.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_slim_finetune2, nbs=64, nms=False, opset=None, optimize=False, optimizer

In [100]:
# Load best model
MODEL_WEIGHTS = "/content/drive/MyDrive/Colab Notebooks/lost_found_project/lostfound_slim_exp/yolov8s_slim_finetune2/weights/best.pt"
best_model = YOLO(MODEL_WEIGHTS)

In [102]:
metrics = best_model.val()

# Print metrics
print("\n" + "="*60)
print("VALIDATION METRICS")
print("="*60)
print(f"mAP50:     {metrics.box.map50:.3f} )")
print(f"mAP50-95:  {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall:    {metrics.box.mr:.3f}")
print("="*60)

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.8±0.2 ms, read: 24.5±10.6 MB/s, size: 38.8 KB)
val: Scanning /content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim/valid/labels.cache... 755 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 755/755 969.9Kit/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 20, len(boxes) = 813. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 48/48 4.2it/s 11.4s
                   all        755        813      0.807       0.75      0.783       0.63
                phones        107        108      0.866      0.852      0.934      0.851
               jackets        151        151      0.982      0.967

In [104]:
import os
import shutil
import glob
import yaml
from ultralytics import YOLO


# Original slimmed dataset (with GT labels)
DATASET_SLIM = "/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim"
DATA_YAML_SLIM = os.path.join(DATASET_SLIM, "data_slim.yaml")

# New dataset root for distillation (teacher pseudo-labels on train)
DATASET_DISTILL = "/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim_distill"
os.makedirs(DATASET_DISTILL, exist_ok=True)


TEACHER_WEIGHTS = "/content/drive/MyDrive/Colab Notebooks/lost_found_project/lostfound_slim_exp/yolov8s_slim_finetune2/weights/best.pt"

print("Using original slim dataset:", DATASET_SLIM)
print("Distilled dataset will be at:", DATASET_DISTILL)

# READ CLASS INFO FROM ORIGINAL SLIM YAML
with open(DATA_YAML_SLIM, "r") as f:
    cfg = yaml.safe_load(f)

names = cfg["names"]
if isinstance(names, dict):
    names = [names[i] for i in sorted(names.keys())]
nc = len(names)

print("\nSlim dataset nc =", nc)
print("Classes:")
for i, n in enumerate(names):
    print(f"  {i}: {n}")

# COPY VAL / TEST AS-IS
for split in ["valid", "val", "test"]:
    src_img_dir = os.path.join(DATASET_SLIM, split, "images")
    src_lbl_dir = os.path.join(DATASET_SLIM, split, "labels")
    if not os.path.exists(src_img_dir):
        continue

    dst_img_dir = os.path.join(DATASET_DISTILL, split, "images")
    dst_lbl_dir = os.path.join(DATASET_DISTILL, split, "labels")
    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    # copy images
    for img_file in os.listdir(src_img_dir):
        src = os.path.join(src_img_dir, img_file)
        dst = os.path.join(dst_img_dir, img_file)
        shutil.copy2(src, dst)

    # copy labels if exist
    if os.path.exists(src_lbl_dir):
        for lbl_file in os.listdir(src_lbl_dir):
            if not lbl_file.endswith(".txt"):
                continue
            src = os.path.join(src_lbl_dir, lbl_file)
            dst = os.path.join(dst_lbl_dir, lbl_file)
            shutil.copy2(src, dst)

print("\nValidation/test splits copied from slim dataset.")

# GENERATE PSEUDO-LABELS FOR TRAIN USING TEACHER
train_img_src_dir = os.path.join(DATASET_SLIM, "train", "images")
train_img_dst_dir = os.path.join(DATASET_DISTILL, "train", "images")
train_lbl_dst_dir = os.path.join(DATASET_DISTILL, "train", "labels")

os.makedirs(train_img_dst_dir, exist_ok=True)
os.makedirs(train_lbl_dst_dir, exist_ok=True)

# Copy train images to distilled dataset
for img_file in os.listdir(train_img_src_dir):
    src = os.path.join(train_img_src_dir, img_file)
    dst = os.path.join(train_img_dst_dir, img_file)
    shutil.copy2(src, dst)

# Use teacher to predict on each train image and save pseudo-label YOLO txt
image_paths = sorted(
    [
        os.path.join(train_img_dst_dir, f)
        for f in os.listdir(train_img_dst_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]
)

print("\nGenerating pseudo-labels from teacher on train images...")
print(f"Total train images: {len(image_paths)}")


for img_path in image_paths:
    results = best_model.predict(
        source=img_path,
        conf=0.5,       # you can adjust this threshold
        imgsz=640,
        verbose=False
    )[0]

    # Results: boxes in absolute coords; we'll convert to normalized xywh
    boxes = results.boxes
    if boxes is None or len(boxes) == 0:
        # no detections: no label file created (image treated as background)
        continue

    # Get image size from results
    h, w = results.orig_shape

    # Prepare label lines
    label_lines = []
    for box in boxes:
        cls_id = int(box.cls[0].item())
        if cls_id < 0 or cls_id >= nc:
            # should not happen if teacher trained on same nc
            continue

        # xyxy in absolute pixels
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        # convert to normalized xywh
        x_center = (x1 + x2) / 2.0 / w
        y_center = (y1 + y2) / 2.0 / h
        bw = (x2 - x1) / w
        bh = (y2 - y1) / h

        label_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n")

    # Write pseudo label file if there is at least one detection
    if label_lines:
        base = os.path.splitext(os.path.basename(img_path))[0]
        lbl_path = os.path.join(train_lbl_dst_dir, base + ".txt")
        with open(lbl_path, "w") as f:
            f.writelines(label_lines)

print("Pseudo-label generation done.")

# WRITE NEW data_distill.yaml
DATA_YAML_DISTILL = os.path.join(DATASET_DISTILL, "data_distill.yaml")

data_distill = {
    "path": DATASET_DISTILL,
    "train": os.path.join(DATASET_DISTILL, "train", "images"),
    "val": os.path.join(DATASET_DISTILL, "valid", "images"),
    "test": os.path.join(DATASET_DISTILL, "test", "images"),
    "nc": nc,
    "names": {i: name for i, name in enumerate(names)},
}

with open(DATA_YAML_DISTILL, "w") as f:
    yaml.safe_dump(data_distill, f, sort_keys=False)

print("\nDistilled data yaml saved to:", DATA_YAML_DISTILL)


Using original slim dataset: /content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim
Distilled dataset will be at: /content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim_distill

Slim dataset nc = 20
Classes:
  0: phones
  1: jackets
  2: glasses
  3: airpods
  4: Scarf
  5: cables
  6: watches
  7: bags(other than backpack)
  8: wallets
  9: card
  10: car-keys
  11: Backpack
  12: laptops
  13: keys
  14: powerbank
  15: colby student card
  16: tablets
  17: chargers
  18: smart-watches
  19: vests

Validation/test splits copied from slim dataset.

Generating pseudo-labels from teacher on train images...
Total train images: 2463
Pseudo-label generation done.

Distilled data yaml saved to: /content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim_distill/data_distill.yaml


In [9]:
import os
import shutil
from ultralytics import YOLO
import torch

student = YOLO("yolov8n.pt")
DATA_YAML_DISTILL = "/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim_distill/data_distill.yaml"
print("\nTraining student model on distilled dataset...")
student_results = student.train(
    data=DATA_YAML_DISTILL,
    epochs=40,
    imgsz=480,
    batch=16,
    workers=4,
    project="lostfound_slim_exp",
    name="yolov8n_student_distilled",
    freeze=10,
)




Training student model on distilled dataset...
Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim_distill/data_distill.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_student_di

In [14]:
# student validation

student_best_path = os.path.join(
    "runs",
    "detect",
    "lostfound_slim_exp",
    "yolov8n_student_distilled",
    "weights",
    "best.pt",
)

DATASET_SLIM = "/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim"
DATA_YAML_SLIM = os.path.join(DATASET_SLIM, "data_slim.yaml")

student_best = YOLO("/content/drive/MyDrive/Colab Notebooks/lost_found_project/lostfound_slim_exp/yolov8n_student_distilled/weights/best.pt")

metrics = student_best.val(
    data=DATA_YAML_SLIM,  # original slim yaml with original labels
    split="val",
)

print("\n" + "=" * 60)
print("STUDENT VALIDATION METRICS (on GT val)")
print("=" * 60)
print(f"mAP50:     {metrics.box.map50:.3f}")
print(f"mAP50-95:  {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall:    {metrics.box.mr:.3f}")
print("=" * 60)


Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.2 ms, read: 0.1±0.0 MB/s, size: 47.8 KB)
val: Scanning /content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim/valid/labels.cache... 755 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 755/755 973.5Kit/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 20, len(boxes) = 813. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 48/48 1.6it/s 30.1s
                   all        755        813      0.711      0.669      0.732       0.57
                phones        107        108      0.832      0.824       0.89

In [ ]:

import os
import shutil
from ultralytics import YOLO
import torch

from ultralytics import YOLO

# Load COCO-pretrained YOLO model nano
model = YOLO("yolov8l.pt")

# Fine-tune on slimmed dataset
results = model.train(
    data="/content/drive/MyDrive/Colab Notebooks/lost_found_project/final_dataset_slim/data_slim.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    workers=4,
    project="lostfound_slim_exp",
    name="yolov8s_slim_finetune",
    freeze = 10
)
